In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

In [1]:
import pandas as pd
import os
import numpy as np

loc = 'train'
dic={'time':-1,'Age':0,'Gender':1,'Height':2,'ICUType':3,'Weight':4,'Albumin':5,\
             'ALP':6,'ALT':7,'AST':8,'Bilirubin':9,'BUN':10,'Cholesterol':11,'Creatinine':12,\
             'DiasABP':13,'FiO2':14,'GCS':15,'Glucose':16,'HCO3':17,'HCT':18,'HR':19,\
             'K':20,'Lactate':21,'Mg':22,'MAP':23,'MechVent':24,'Na':25,'NIDiasABP':26,\
             'NIMAP':27,'NISysABP':28,'PaCO2':29,'PaO2':30,'pH':31,'Platelets':32,'RespRate':33,\
             'SaO2':34,'SysABP':35,'Temp':36,'TroponinI':37,'TroponinT':38,'Urine':39,'WBC':40}

def handle(loc):
    dataPath = '../Multivariate-Time-Series-Imputation-with-Generative-Adversarial-Networks/set-a/{}/'.format(loc)
    labelFile = '../Multivariate-Time-Series-Imputation-with-Generative-Adversarial-Networks/set-a/{}/list.txt'.format(loc)
    data_lis = []
    with open(labelFile, "r") as f:
        line_num = 0
        for line in f.readlines():
            if line_num!=0:
                words = line.strip().split(',') 
                if os.path.isfile(os.path.join(dataPath, words[0]+".txt")):
                    data_lis.append((words[0]+".txt", words[len(words)-1]))
            line_num += 1   

    print(len(data_lis))        
    sta = []
    dyn = []
    lis = [x for x,y in dic.items()]
    print(lis)
    for fileName, label in data_lis:
        with open(os.path.join(dataPath, fileName)) as f:
            #read_csv is DataFrame, need to be transformed into ndarray
            count=0
            age=gender=height=icutype=weight=np.nan
            lastTime=0
            totalData=[]
            for line in f.readlines():
                if count > 1:
                    words=line.split(",")
                    timestamp=words[0]
                    feature=words[1]
                    value=words[2]

                    # -1 is missing value
                    if timestamp == "00:00":
                        if feature=='Age':
                            age=float(value)
                            if age == -1: age = np.nan
                        if feature=='Gender':
                            gender=int(value)
                            if gender == -1: gender = np.nan
                        if feature=='Height':
                            height=float(value)
                            if height == -1: height = np.nan
                        if feature == 'ICUType':
                            icutype=int(value)
                            if icutype == -1: icutype = np.nan
                        if feature=='Weight':
                            weight=float(value)
                            if weight == -1: weight = np.nan
                    else:
                        if timestamp!=lastTime:
                            data=[np.nan]*42
                            hourandminute=timestamp.split(":")
                            data[0]=float(hourandminute[0])*60+float(hourandminute[1])
                            """
                            data[1]=float(age)
                            data[2]=float(gender)
                            data[3]=float(height)
                            data[4]=float(icutype)
                            data[5]=float(weight)
                            """
                            if float(value)!=-1:
                                data[dic[feature]]=float(value)
                            if feature == 'Weight':
                                assert float(value) == float(weight)
                            totalData.append(data)
                        else:
                            totalData[len(totalData)-1][dic[feature]]=float(value)
                    lastTime=timestamp

                count+=1
        sta.append([age,gender,height,icutype,weight,int(label)])
        d = np.array(totalData)
        #for i in range(len(d)-1, 0, -1):
        #    d[i][0]-=d[i-1][0]
        d = pd.DataFrame(d, columns=lis)
        dyn.append(d.drop(columns=['Age', 'Gender','Height','ICUType','Weight', 'WBC']))
        
    sta = pd.DataFrame(np.array(sta), columns=['Age', 'Gender','Height','ICUType','Weight', 'Label'])
    return sta,dyn


In [1]:
sta,dyn=handle('train')

NameError: name 'handle' is not defined

In [1]:
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_parquet("/home/stelios-pc/Downloads/iV2Ip (2).parquet")
df.drop(columns=["bag_file", "line_of_sight"], inplace=True)
df = df.reset_index()
df.drop(columns=("timestamp"), inplace=True)
df.rename(columns={"times[s]" : "timestamp"}, inplace=True)
df


,time[s],serving_cell_rsrp_1,serving_cell_rsrq_1,serving_cell_rssi_1,serving_cell_snr_1,datarate_client,jitter_client,port_local_client,port_remote_client,target_DL,...,orientation_z,orientation_w,twist_linear_x,twist_linear_y,twist_linear_z,twist_angular_x,twist_angular_y,twist_angular_z,distance_to_bs,obstacles_sum
0,1.639485e+09,-121.0,-13.0,-86.0,9.0,7620000.0,0.002451,55676.0,5204.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.639485e+09,-120.4,-13.0,-86.2,9.0,7380000.0,0.001974,55676.0,5204.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.639485e+09,-121.0,-13.8,-86.2,8.8,7320000.0,0.001735,55676.0,5204.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.639485e+09,-121.0,-13.8,-86.2,8.8,6580000.0,0.001478,55676.0,5204.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.639485e+09,-120.6,-13.8,-86.4,9.0,6550000.0,0.001616,55676.0,5204.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53194,1.639664e+09,-124.2,-15.2,-88.8,7.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53195,1.639664e+09,-127.0,-16.4,-88.4,6.8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53196,1.639664e+09,-126.8,-16.2,-88.4,7.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53197,1.639664e+09,-126.2,-16.0,-88.8,7.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from general.missingprocessor import Processor

import pickle

seq_len = 24
types = ["continuous" for i in range(len(df.columns))]
P = Processor(types)
ori_data = P.fit_transform(df)
temp_data = [ori_data[i:i + seq_len] for i in range(0, len(ori_data) - seq_len)]
from fastNLP import DataSet
dataset = DataSet({"seq_len": [seq_len] * len(temp_data), "dyn": temp_data, "sta":[0]*len(temp_data)})
dic = {
    "train_set": dataset,
    "dynamic_processor": P,
    "static_processor": Processor([])
}
print(len(temp_data))
with open("./data/{}.pkl".format("ericsson"), "wb") as f:
    pickle.dump(dic, f)

time[s] 2 0.2661704167371567 continuous None
[1.63966367e+09] [1.63948505e+09]
serving_cell_rsrp_1 2 0.2661704167371567 continuous None
[-65.66666667] [-133.5]
serving_cell_rsrq_1 2 0.2661704167371567 continuous None
[-4.6] [-37.66666667]
serving_cell_rssi_1 2 0.26618921408297147 continuous None
[-50.33333333] [-89.]
serving_cell_snr_1 2 0.26618921408297147 continuous None
[23.8] [-10.]
datarate_client 2 0.19237203706836595 continuous None
[2.94e+08] [0.]
jitter_client 2 0.19237203706836595 continuous None
[30.617] [0.]
all values are integer
port_local_client 2 0.19237203706836595 int None
[60344.] [33275.]
all values are integer
port_remote_client 2 0.19237203706836595 int None
[5204.] [5201.]
all values are integer
target_DL 2 0.5611195699167277 int None
[80.] [20.]
throughput_DL 2 0.5611195699167277 continuous None
[67.1] [0.]
jitter_DL 2 0.5611195699167277 continuous None
[0.213047] [9.6e-05]
all values are integer
device_id_client 2 0.5611195699167277 int None
[0.] [0.]
datarate_

In [3]:
sta

,Age,Gender,Height,ICUType,Weight,Label
0,54.0,0.0,NaN,4.0,NaN,0.0
1,76.0,1.0,175.3,2.0,76.0,0.0
2,44.0,0.0,NaN,3.0,56.7,0.0
3,68.0,1.0,180.3,3.0,84.6,0.0
4,88.0,0.0,NaN,3.0,NaN,0.0
...,...,...,...,...,...,...
3589,39.0,1.0,177.8,3.0,87.1,0.0
3590,58.0,0.0,NaN,1.0,102.8,0.0
3591,90.0,1.0,NaN,3.0,78.0,0.0
3592,56.0,0.0,170.2,2.0,131.2,0.0


In [4]:
dyn[0]

,time,Albumin,ALP,ALT,AST,Bilirubin,BUN,Cholesterol,Creatinine,DiasABP,...,PaO2,pH,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine
0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19.0,NaN,NaN,35.1,NaN,NaN,900.0,NaN
1,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19.0,NaN,NaN,35.6,NaN,NaN,60.0,NaN
2,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,30.0,NaN
3,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,170.0,NaN
4,188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20.0,NaN,NaN,37.8,NaN,NaN,60.0,NaN
6,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,337.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,170.0,NaN
8,457.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15.0,NaN,NaN,38.1,NaN,NaN,120.0,NaN
9,517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,80.0,NaN


In [34]:
from general.missingprocessor import Processor
d_P = Processor(dyn_types, use_pri='time[s]')
s_P = Processor(sta_types)
d_P.fit(df)
# s_P.fit([])

time[s] 1 0.2661654135338346 continuous None
[1.63966367e+09] [0.]
serving_cell_rsrp_1 1 0.2661704167371567 continuous None
[-65.66666667] [-133.5]
serving_cell_rsrq_1 1 0.2661704167371567 continuous None
[-4.6] [-37.66666667]
serving_cell_rssi_1 1 0.26618921408297147 continuous None
[-50.33333333] [-89.]
serving_cell_snr_1 1 0.26618921408297147 continuous None
[23.8] [-10.]
datarate_client 1 0.19237203706836595 continuous None
[2.94e+08] [0.]
jitter_client 1 0.19237203706836595 continuous None
[30.617] [0.]
all values are integer
port_local_client 1 0.19237203706836595 int None
[60344.] [33275.]
all values are integer
port_remote_client 1 0.19237203706836595 int None
[5204.] [5201.]
all values are integer
target_DL 1 0.5611195699167277 int None
[80.] [20.]
throughput_DL 1 0.5611195699167277 continuous None
[67.1] [0.]
jitter_DL 1 0.5611195699167277 continuous None
[0.213047] [9.6e-05]
all values are integer
device_id_client 1 0.5611195699167277 int None
[0.] [0.]
datarate_server 1 0.1

In [40]:
p = d_P.transform(df)

In [52]:
p[5]

array([[0.99989106],
       [0.99989106],
       [0.99989106],
       ...,
       [1.        ],
       [1.        ],
       [1.        ]])

In [37]:
def build_dataset(dyn, seq_len):
    # s = s_P.transform(sta)
    d_lis=[d_P.transform(ds) for ds in dyn]
    d = [x[0].tolist() for x in d_lis]
    lag = [x[1].tolist() for x in d_lis]
    mask = [x[2].tolist() for x in d_lis]
    times = [x[-1].tolist() for x in d_lis]
    priv = [x[3].tolist() for x in d_lis]
    nex = [x[4].tolist() for x in d_lis]
    # label = [float(x[-2]) for x in s] 
    from fastNLP import DataSet
    dataset = DataSet({"seq_len": seq_len, 
                       "dyn": d, "lag":lag, "mask": mask,
                     "times":times, "priv":priv, "nex":nex
                      })
    return dataset

dataset = build_dataset(df, seq_len)

AttributeError: 'str' object has no attribute 'values'

In [8]:
test_sta, test_dyn = handle('test')
test_seq_len = [len(x) for x in test_dyn]
test_sta["seq_len"]=np.array(test_seq_len)
val_set = build_dataset(test_sta, test_dyn, test_seq_len)

400
['time', 'Age', 'Gender', 'Height', 'ICUType', 'Weight', 'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT', 'HR', 'K', 'Lactate', 'Mg', 'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 'PaO2', 'pH', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC']


In [9]:
finaldic = {
    "train_set": dataset,
    'raw_set': (sta,dyn),
    'test_set': (test_sta,test_dyn),
    'val_set': val_set,
    "dynamic_processor": d_P,
    "static_processor":s_P
}
import pickle
from utils.general import make_sure_path_exists
make_sure_path_exists("./data/physio_data/")
with open("./data/physio_data/full2012.pkl", "wb") as f:
    pickle.dump(finaldic, f)
